## Import the libraries

In [52]:
!pip3 install dataframe_image
!pip3 install pycaret
!pip3 install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
%pip install -U scikit-learn pandas numpy matplotlib "pycaret==3.3.2"


  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.7-cp310-cp310-win_amd64.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import resample
from pycaret.classification import *
#import dataframe_image as dfi
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import os
from pathlib import Path


In [55]:
!python --version


Python 3.10.11


##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Method.

#### Setup the model creation with pycaret for Long Method detection

In [56]:
df = pd.read_csv('dataset_long_method.csv')
df

,raw_sloc,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,smell_label
0,3,9,3,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-long-method
1,13,0,1,0,2,3,3,2.000000,4.754888,0.500000,2.377444,0.132080,0.001585,non-long-method
2,11,0,0,0,2,3,3,2.000000,4.754888,0.500000,2.377444,0.132080,0.001585,non-long-method
3,5,0,0,0,2,3,3,2.000000,4.754888,0.500000,2.377444,0.132080,0.001585,non-long-method
4,4,12,2,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,non-long-method
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,2,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,long-method
826,2,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,long-method
827,2,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,long-method
828,2,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,long-method


In [57]:
# # Balance dataset using sklearn resampling
# class_counts = df['smell_label'].value_counts()
# print('Class distribution before balancing:')
# print(class_counts)

# max_count = class_counts.max()
# balanced_parts = [
#     resample(group, replace=True, n_samples=max_count, random_state=42)
#     for _, group in df.groupby('smell_label')
# ]

# df = (pd.concat(balanced_parts)
#       .sample(frac=1, random_state=42)
#       .reset_index(drop=True))

# print('\nClass distribution after balancing:')
# print(df['smell_label'].value_counts())
# df.head()


In [58]:
# Configuração otimizada para evitar overprocessing
s = setup(
    data=df,
    target='smell_label',
    session_id=42,
    fold=10,
    fix_imbalance=True,
    fix_imbalance_method=SMOTE(k_neighbors=3, random_state=42),  # k menor para melhor generalização
    feature_selection=False,  # Desabilitar feature selection automática
    remove_multicollinearity=False,  # Manter features correlacionadas
    normalize=True,
    transformation=False,  # Reduzir transformation
    pca=False,  # Remover PCA para manter features originais
    verbose=False
    )

In [59]:
df = get_config('X')
df

print("Shape do dataset processado:", df.shape)
print("\nEstatísticas das features processadas:")
print(df.describe())

# Verificar variância das features
print("\nVariância das features:")
print(df.var().sort_values())

# Verificar correlação
print("\nMatriz de correlação (primeiras 5 features):")
print(df.iloc[:, :5].corr())

# Verificar distribuição das classes depois do processamento
y_train = get_config('y_train')
print("\nDistribuição das classes no treino (após SMOTE):")
print(y_train.value_counts())

Shape do dataset processado: (830, 13)

Estatísticas das features processadas:
         raw_sloc   raw_multi   raw_blank  raw_single_comments  hal_func_N2  \
count  830.000000  830.000000  830.000000           830.000000   830.000000   
mean    26.091566    9.037349    2.033735             0.027711    25.026506   
std     42.112364   23.227408    6.710726             0.171429    60.921586   
min      2.000000    0.000000    0.000000             0.000000     0.000000   
25%      3.000000    0.000000    0.000000             0.000000     0.000000   
50%      5.000000    0.000000    0.000000             0.000000     1.000000   
75%     52.000000    8.000000    2.000000             0.000000    22.000000   
max    480.000000  263.000000  105.000000             2.000000  1003.000000   

       hal_func_vocabulary  hal_func_length  hal_func_calculated_length  \
count           830.000000       830.000000                  830.000000   
mean             17.918072        39.067470                

# Comparar modelos com configuração otimizada para evitar warnings
# Usar n_select menor para focar nos melhores modelos


In [60]:
top5_models = compare_models(n_select=13, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.9174,0.9139,0.9174,0.9202,0.9159,0.8147,0.8204,0.0050
lr,Logistic Regression,0.9174,0.9308,0.9174,0.9227,0.9153,0.8130,0.8223,0.0060
lightgbm,Light Gradient Boosting Machine,0.9123,0.9623,0.9123,0.9144,0.9115,0.8065,0.8100,0.0410
qda,Quadratic Discriminant Analysis,0.9122,0.9346,0.9122,0.9146,0.9110,0.8043,0.8090,0.0050
ridge,Ridge Classifier,0.9123,0.9284,0.9123,0.9240,0.9082,0.7967,0.8154,0.0050
lda,Linear Discriminant Analysis,0.9071,0.9231,0.9071,0.9186,0.9027,0.7846,0.8034,0.2390
knn,K Neighbors Classifier,0.9037,0.9278,0.9037,0.9099,0.9015,0.7833,0.7933,0.0100
svm,SVM - Linear Kernel,0.8698,0.9033,0.8698,0.9027,0.8630,0.7361,0.7570,0.0040
dummy,Dummy Classifier,0.3580,0.5000,0.3580,0.1282,0.1888,0.0000,0.0000,0.0040


In [61]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongMethod_results.latex',index=False)
df3.to_csv('LongMethod_results.csv',index=False)
#dfi.export(df2, 'Bin_Blob_FS1.png')

In [62]:
from sklearn.metrics import classification_report
from pycaret.classification import predict_model

# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []

# Iterar sobre os modelos
for model in top5_models:
    # Gerar predições com o modelo individual
    predictions = predict_model(model)

    # Obter nome do modelo
    model_name = model.__class__.__name__

    # Gerar relatório por classe
    report = classification_report(
        predictions['smell_label'],
        predictions['prediction_label'],
        output_dict=True
    )

    # Armazenar o relatório
    reports[model_name] = report

    # Exibir e armazenar métricas por classe
    print(f"\n📊 Classification Report for {model_name}")
    for smell_class in ['long-method']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")

            # Armazenar para eventual DataFrame
            data.append({
                "Model": model_name,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.9277,0.9372,0.9277,0.9282,0.9269,0.8394,0.8416



📊 Classification Report for GaussianNB

Smell: long-method
  Accuracy: 0.93
  Precision: 0.94
  Recall: 0.85
  F1-score: 0.89


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9357,0.9682,0.9357,0.9365,0.9350,0.8573,0.8595



📊 Classification Report for LogisticRegression

Smell: long-method
  Accuracy: 0.94
  Precision: 0.95
  Recall: 0.87
  F1-score: 0.91


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9157,0.9617,0.9157,0.9155,0.9156,0.8159,0.8160


...


📊 Classification Report for KNeighborsClassifier

Smell: long-method
  Accuracy: 0.93
  Precision: 0.95
  Recall: 0.84
  F1-score: 0.89


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,SVM - Linear Kernel,0.7871,0,0.7871,0.8469,0.7912,0.5834,0.6237



📊 Classification Report for SGDClassifier

Smell: long-method
  Accuracy: 0.79
  Precision: 0.63
  Recall: 0.96
  F1-score: 0.76


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.3574,0.5000,0.3574,0.1278,0.1882,0.0000,0.0000



📊 Classification Report for DummyClassifier

Smell: long-method
  Accuracy: 0.36
  Precision: 0.36
  Recall: 1.00
  F1-score: 0.53


In [63]:
# Criar um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("longmethod_classification_results.csv", index=False)

In [64]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('longmethod_performance.latex',index=False)
df3.to_csv('longmethod_performance.csv',index=False)

In [65]:
top_models = compare_models(n_select=13, sort='f1')  
best = top_models[0]
best


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.9174,0.9139,0.9174,0.9202,0.9159,0.8147,0.8204,0.0050
lr,Logistic Regression,0.9174,0.9308,0.9174,0.9227,0.9153,0.8130,0.8223,0.0050
lightgbm,Light Gradient Boosting Machine,0.9123,0.9623,0.9123,0.9144,0.9115,0.8065,0.8100,0.0450
qda,Quadratic Discriminant Analysis,0.9122,0.9346,0.9122,0.9146,0.9110,0.8043,0.8090,0.0050
ridge,Ridge Classifier,0.9123,0.9284,0.9123,0.9240,0.9082,0.7967,0.8154,0.0050
lda,Linear Discriminant Analysis,0.9071,0.9231,0.9071,0.9186,0.9027,0.7846,0.8034,0.2420
knn,K Neighbors Classifier,0.9037,0.9278,0.9037,0.9099,0.9015,0.7833,0.7933,0.0100
svm,SVM - Linear Kernel,0.8698,0.9033,0.8698,0.9027,0.8630,0.7361,0.7570,0.0050
dummy,Dummy Classifier,0.3580,0.5000,0.3580,0.1282,0.1888,0.0000,0.0000,0.0050


GaussianNB(priors=None, var_smoothing=1e-09)

#### Ensemble the top-5 models

In [66]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9153,0.0000,0.9153,0.9174,0.9136,0.8091,0.8144
1,0.9655,0.0000,0.9655,0.9673,0.9651,0.9238,0.9265
2,0.8621,0.0000,0.8621,0.8640,0.8583,0.6886,0.6971
3,0.9138,0.0000,0.9138,0.9241,0.9107,0.8033,0.8193
4,0.9310,0.0000,0.9310,0.9378,0.9292,0.8443,0.8547
5,0.9655,0.0000,0.9655,0.9673,0.9651,0.9238,0.9265
6,0.9310,0.0000,0.9310,0.9378,0.9292,0.8443,0.8547
7,0.9138,0.0000,0.9138,0.9241,0.9107,0.8033,0.8193
8,0.8966,0.0000,0.8966,0.9005,0.8934,0.7597,0.7696


In [67]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9357,0,0.9357,0.9365,0.9350,0.8573,0.8595


In [68]:
result = pull()

In [69]:
result.to_csv('LongMethod_ensemble.csv')

#### Plot the ensemble model

#### Finalize and save the ensemble model

In [70]:
candidate = best  # ou tuned / best

try:
    final_best = finalize_model(candidate)
except Exception as e:
    print("[Aviso] finalize_model falhou. Reaplicando workaround com fix_imbalance=False...")
    set_config('fix_imbalance', False)   # garante que o PyCaret finalize sem reamostragem
    final_best = finalize_model(candidate)

final_best


Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['raw_sloc', 'raw_multi',
                                             'raw_blank', 'raw_single_comments',
                                             'hal_func_N2',
                                             'hal_func_vocabulary',
                                             'hal_func_length',
                                             'hal_func_calculated_length',...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=FixImbalancer(estimator=SMOTE(k_neighbors=3,
                                                                              random_state=42,
                                                                              sampling_strategy='auto')))),
                ('normalize',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                                                               with_std=True))),
                ('actual_estimator',
                 GaussianNB(priors=None, var_smoothing=1e-09))],
         verbose=False)

In [71]:
save_model(final_best, 'long_method')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['raw_sloc', 'raw_multi',
                                              'raw_blank', 'raw_single_comments',
                                              'hal_func_N2',
                                              'hal_func_vocabulary',
                                              'hal_func_length',
                                              'hal_func_calculated_length',...
                  TransformerWrapper(exclude=None, include=None,
                                     transformer=FixImbalancer(estimator=SMOTE(k_neighbors=3,
                                                                               random_state=42

#### Salvar os 5 melhores modelos individualmente

In [72]:
# Criar um resumo dos modelos salvos com suas métricas
model_summary = []

for i, model in enumerate(top5_models[:5]):
    model_name = model.__class__.__name__
    
    # Gerar predições para obter métricas
    predictions = predict_model(model)
    
    # Gerar relatório de classificação
    report = classification_report(
        predictions['smell_label'],
        predictions['prediction_label'],
        output_dict=True
    )
    
    # Extrair métricas para long-method
    if 'long-method' in report:
        model_summary.append({
            'Model': model_name,
            'Rank': i + 1,
            'F1_Score': report['long-method']['f1-score'],
            'Precision': report['long-method']['precision'],
            'Recall': report['long-method']['recall'],
            'Accuracy': report['accuracy']
        })

# Criar DataFrame com o resumo
summary_df = pd.DataFrame(model_summary)

# Exibir resumo
print("📊 Resumo dos 5 melhores modelos:")
print(summary_df.to_string(index=False))

# Salvar resumo em CSV
summary_df.to_csv('long_method_top5_models_summary.csv', index=False)
print(f"\n📁 Resumo salvo como: long_method_top5_models_summary.csv")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.9277,0.9372,0.9277,0.9282,0.9269,0.8394,0.8416


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.9357,0.9682,0.9357,0.9365,0.9350,0.8573,0.8595


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9157,0.9617,0.9157,0.9155,0.9156,0.8159,0.8160


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.9357,0.9360,0.9357,0.9359,0.9352,0.8580,0.8592


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Ridge Classifier,0.9116,0,0.9116,0.9223,0.9083,0.7965,0.8135


📊 Resumo dos 5 melhores modelos:
                        Model  Rank  F1_Score  Precision   Recall  Accuracy
                   GaussianNB     1  0.894118   0.938272 0.853933  0.927711
           LogisticRegression     2  0.905882   0.950617 0.865169  0.935743
               LGBMClassifier     3  0.881356   0.886364 0.876404  0.915663
QuadraticDiscriminantAnalysis     4  0.906977   0.939759 0.876404  0.935743
              RidgeClassifier     5  0.858974   1.000000 0.752809  0.911647

📁 Resumo salvo como: long_method_top5_models_summary.csv


In [73]:
# Salvar os 5 melhores modelos individualmente
print("Salvando os 5 melhores modelos para Long Method detection...")

# Lista para armazenar os modelos finalizados e seus nomes
finalized_models = []

# Iterar sobre os 5 melhores modelos
for i, model in enumerate(top5_models[:5]):
    # Obter nome do modelo
    model_name = model.__class__.__name__
    print(f"\n{i+1}. Processando modelo: {model_name}")
    
    # Finalizar o modelo
    try:
        final_model = finalize_model(model)
        
        # Criar nome descritivo para o arquivo
        filename = f"{model_name.lower()}_long_method"
        
        # Salvar o modelo
        save_model(final_model, filename)
        print(f"   Modelo salvo como: {filename}.pkl")
        
        # Adicionar à lista de modelos finalizados
        finalized_models.append({
            'name': model_name,
            'filename': filename,
            'model': final_model
        })
        
    except Exception as e:
        print(f"   Erro ao salvar modelo {model_name}: {str(e)}")
        # Tentar com workaround
        try:
            set_config('fix_imbalance', False)
            final_model = finalize_model(model)
            filename = f"long_method_{model_name.lower()}_f1_fixed"
            save_model(final_model, filename)
            print(f"   Modelo salvo com workaround como: {filename}.pkl")
        except Exception as e2:
            print(f"   Falha ao salvar mesmo com workaround: {str(e2)}")

print(f"\n✅ {len(finalized_models)} modelos salvos com sucesso!")
print("\nModelos disponíveis:")
for model_info in finalized_models:
    print(f"  - {model_info['name']}: {model_info['filename']}.pkl")

Salvando os 5 melhores modelos para Long Method detection...

1. Processando modelo: GaussianNB
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: gaussiannb_long_method.pkl

2. Processando modelo: LogisticRegression
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: logisticregression_long_method.pkl

3. Processando modelo: LGBMClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: lgbmclassifier_long_method.pkl

4. Processando modelo: QuadraticDiscriminantAnalysis
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: quadraticdiscriminantanalysis_long_method.pkl

5. Processando modelo: RidgeClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: ridgeclassifier_long_method.pkl

✅ 5 modelos salvos com sucesso!

Modelos disponíveis:
  - GaussianNB: gaussiannb_long_method.pkl
  - LogisticRegression: logisticregression_long_method.pkl
  - LGBMClassifier: lgbmc